In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
from sklearn.metrics import roc_auc_score
# np.random.seed(2024)
# torch.manual_seed(2024)
import pdb

# from dataset import load_data
from matrix_factorization_product import MF_KBIPS_Gau,MF_AKBIPS_Gau,MF_WKBIPS_Gau,MF_KBDR_Gau,MF_AKBDR_Gau,MF_WKBDR_Gau
from matrix_factorization_product import MF_KBIPS_Exp,MF_AKBIPS_Exp,MF_WKBIPS_Exp,MF_KBDR_Exp,MF_AKBDR_Exp,MF_WKBDR_Exp

from utils import gini_index, ndcg_func, get_user_wise_ctr, rating_mat_to_sample, binarize, shuffle, minU,recall_func, precision_func
mse_func = lambda x,y: np.mean((x-y)**2)
acc_func = lambda x,y: np.sum(x == y) / len(x)

dataset_name = "kuai"

import pandas as pd

In [ ]:
rdf_train = np.array(pd.read_table("./data/kuai/user.txt", header = None, sep = ','))     
rdf_test = np.array(pd.read_table("./data/kuai/random.txt", header = None, sep = ','))
rdf_train_new = np.c_[rdf_train, np.ones(rdf_train.shape[0])]
rdf_test_new = np.c_[rdf_test, np.zeros(rdf_test.shape[0])]
rdf = np.r_[rdf_train_new, rdf_test_new]
    
rdf = rdf[np.argsort(rdf[:, 0])]
c = rdf.copy()
for i in range(rdf.shape[0]):
    if i == 0:
        c[:, 0][i] = i
        temp = rdf[:, 0][0]
    else:
        if c[:, 0][i] == temp:
            c[:, 0][i] = c[:, 0][i-1]
        else:
            c[:, 0][i] = c[:, 0][i-1] + 1
        temp = rdf[:, 0][i]
    
c = c[np.argsort(c[:, 1])]
d = c.copy()
for i in range(rdf.shape[0]):
    if i == 0:
        d[:, 1][i] = i
        temp = c[:, 1][0]
    else:
        if d[:, 1][i] == temp:
            d[:, 1][i] = d[:, 1][i-1]
        else:
            d[:, 1][i] = d[:, 1][i-1] + 1
        temp = c[:, 1][i]

y_train = d[:, 2][d[:, 3] == 1]
y_test = d[:, 2][d[:, 3] == 0]
x_train = d[:, :2][d[:, 3] == 1]
x_test = d[:, :2][d[:, 3] == 0]
    
num_user = x_train[:,0].max() + 1
num_item = x_train[:,1].max() + 1

    
y_train = binarize(y_train, 2)
y_test = binarize(y_test, 2)

num_user = int(num_user)
num_item = int(num_item)
print("# user: {}, # item: {}".format(num_user, num_item))

### KBIPS

In [ ]:
mf_kbips_gau = MF_KBIPS_Gau(num_user, num_item)
mf_kbips_gau.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_kbips_gau.fit(x_train, y_train,  y_ips,
    lr1=1e-2,
    lamb1=5e-5,
    lr2=1e-2,
    lamb2=1e-1,
    gamma = 1,
    batch_size=2048,
    J = 3,
    C = 1e-2,
    tol=1e-5)
test_pred = mf_kbips_gau.predict(x_test)
mse_mfkbips = mse_func(y_test, test_pred)
auc_mfkbips = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_kbips_gau, x_test, y_test, top_k_list = [20, 50])
recall_res = recall_func(mf_kbips_gau, x_test, y_test, top_k_list = [20, 50])
precision_res = precision_func(mf_kbips_gau, x_test, y_test, top_k_list = [20, 50])

print("***"*5 + "[MF-KBIPS-Gau]" + "***"*5)
print("[MF-KBIPS-Gau] test mse:", mse_mfkbips)
print("[MF-KBIPS-Gau] test auc:", auc_mfkbips)
print("[MF-KBIPS-Gau] ndcg@20:{:.6f}, ndcg@50:{:.6f}".format(
        np.mean(ndcg_res["ndcg_20"]), np.mean(ndcg_res["ndcg_50"])))
print("[MF-KBIPS-Gau] recall@20:{:.6f}, recall@50:{:.6f}".format(
        np.mean(recall_res["recall_20"]), np.mean(recall_res["recall_50"])))
print("[MF-KBIPS-Gau] precision@20:{:.6f}, precision@50:{:.6f}".format(
        np.mean(precision_res["precision_20"]), np.mean(precision_res["precision_50"])))
print("[MF-KBIPS-Gau] f1@20:{:.6f}, f1@50:{:.6f}".format(
        2 * (np.mean(precision_res["precision_20"]) * np.mean(recall_res["recall_20"])) / (np.mean(precision_res["precision_20"]) + np.mean(recall_res["recall_20"])), 
        2 * (np.mean(precision_res["precision_50"]) * np.mean(recall_res["recall_50"])) / (np.mean(precision_res["precision_50"]) + np.mean(recall_res["recall_50"]))))
print("***"*5 + "[MF-KBIPS-Gau]" + "***"*5)

In [ ]:
mf_kbips_exp = MF_KBIPS_Exp(num_user, num_item)
mf_kbips_exp.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_kbips_exp.fit(x_train, y_train,  y_ips,
    lr1=1e-2,
    lamb1=5e-5,
    lr2=1e-2,
    lamb2=1e-1,
    gamma = 1,
    batch_size=2048,
    J = 3,
    C = 1e-2,
    tol=1e-5)
test_pred = mf_kbips_exp.predict(x_test)
mse_mfkbips = mse_func(y_test, test_pred)
auc_mfkbips = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_kbips_exp, x_test, y_test, top_k_list = [20, 50])
recall_res = recall_func(mf_kbips_exp, x_test, y_test, top_k_list = [20, 50])
precision_res = precision_func(mf_kbips_exp, x_test, y_test, top_k_list = [20, 50])

print("***"*5 + "[MF-KBIPS-Exp]" + "***"*5)
print("[MF-KBIPS-Exp] test mse:", mse_mfkbips)
print("[MF-KBIPS-Exp] test auc:", auc_mfkbips)
print("[MF-KBIPS-Exp] ndcg@20:{:.6f}, ndcg@50:{:.6f}".format(
        np.mean(ndcg_res["ndcg_20"]), np.mean(ndcg_res["ndcg_50"])))
print("[MF-KBIPS-Exp] recall@20:{:.6f}, recall@50:{:.6f}".format(
        np.mean(recall_res["recall_20"]), np.mean(recall_res["recall_50"])))
print("[MF-KBIPS-Exp] precision@20:{:.6f}, precision@50:{:.6f}".format(
        np.mean(precision_res["precision_20"]), np.mean(precision_res["precision_50"])))
print("[MF-KBIPS-Exp] f1@20:{:.6f}, f1@50:{:.6f}".format(
        2 * (np.mean(precision_res["precision_20"]) * np.mean(recall_res["recall_20"])) / (np.mean(precision_res["precision_20"]) + np.mean(recall_res["recall_20"])), 
        2 * (np.mean(precision_res["precision_50"]) * np.mean(recall_res["recall_50"])) / (np.mean(precision_res["precision_50"]) + np.mean(recall_res["recall_50"]))))
print("***"*5 + "[MF-KBIPS-Exp]" + "***"*5)

### AKBIPS

In [ ]:
mf_akbips_Gau = MF_AKBIPS_Gau(num_user, num_item)
mf_akbips_Gau.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_akbips_Gau.fit(x_train, y_train,  y_ips,
    lr1=3e-2,
    lamb1=5e-5,
    lr2=1e-2,
    lamb2=1e-1,
    lr3=3e-2,
    lamb3=1e-4,
    gamma = 1,
    batch_size=2048,
    num_w_epo = 3,
    J = 3,
    C = 1e-2,
    tol=1e-5)
test_pred = mf_akbips_Gau.predict(x_test)
mse_mfakbips = mse_func(y_test, test_pred)
auc_mfakbips = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_akbips_Gau, x_test, y_test, top_k_list = [20, 50])
recall_res = recall_func(mf_akbips_Gau, x_test, y_test, top_k_list = [20, 50])
precision_res = precision_func(mf_akbips_Gau, x_test, y_test, top_k_list = [20, 50])

print("***"*5 + "[MF-AKBIPS-Gau]" + "***"*5)
print("[MF-AKBIPS-Gau] test mse:", mse_mfakbips)
print("[MF-AKBIPS-Gau] test auc:", auc_mfakbips)
print("[MF-AKBIPS-Gau] ndcg@20:{:.6f}, ndcg@50:{:.6f}".format(
        np.mean(ndcg_res["ndcg_20"]), np.mean(ndcg_res["ndcg_50"])))
print("[MF-AKBIPS-Gau] recall@20:{:.6f}, recall@50:{:.6f}".format(
        np.mean(recall_res["recall_20"]), np.mean(recall_res["recall_50"])))
print("[MF-AKBIPS-Gau] precision@20:{:.6f}, precision@50:{:.6f}".format(
        np.mean(precision_res["precision_20"]), np.mean(precision_res["precision_50"])))
print("[MF-AKBIPS-Gau] f1@20:{:.6f}, f1@50:{:.6f}".format(
        2 * (np.mean(precision_res["precision_20"]) * np.mean(recall_res["recall_20"])) / (np.mean(precision_res["precision_20"]) + np.mean(recall_res["recall_20"])), 
        2 * (np.mean(precision_res["precision_50"]) * np.mean(recall_res["recall_50"])) / (np.mean(precision_res["precision_50"]) + np.mean(recall_res["recall_50"]))))
print("***"*5 + "[MF-AKBIPS-Gau]" + "***"*5)

In [ ]:
mf_akbips_exp = MF_AKBIPS_Exp(num_user, num_item)
mf_akbips_exp.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_akbips_exp.fit(x_train, y_train,  y_ips,
    lr1=3e-2,
    lamb1=5e-5,
    lr2=1e-2,
    lamb2=1e-1,
    lr3=3e-2,
    lamb3=1e-4,
    gamma = 1,
    batch_size=2048,
    num_w_epo = 3,
    J = 3,
    C = 1e-2,
    tol=1e-5)
test_pred = mf_akbips_exp.predict(x_test)
mse_mfakbips = mse_func(y_test, test_pred)
auc_mfakbips = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_akbips_exp, x_test, y_test, top_k_list = [20, 50])
recall_res = recall_func(mf_akbips_exp, x_test, y_test, top_k_list = [20, 50])
precision_res = precision_func(mf_akbips_exp, x_test, y_test, top_k_list = [20, 50])

print("***"*5 + "[MF-AKBIPS-Exp]" + "***"*5)
print("[MF-AKBIPS-Exp] test mse:", mse_mfakbips)
print("[MF-AKBIPS-Exp] test auc:", auc_mfakbips)
print("[MF-AKBIPS-Exp] ndcg@20:{:.6f}, ndcg@50:{:.6f}".format(
        np.mean(ndcg_res["ndcg_20"]), np.mean(ndcg_res["ndcg_50"])))
print("[MF-AKBIPS-Exp] recall@20:{:.6f}, recall@50:{:.6f}".format(
        np.mean(recall_res["recall_20"]), np.mean(recall_res["recall_50"])))
print("[MF-AKBIPS-Exp] precision@20:{:.6f}, precision@50:{:.6f}".format(
        np.mean(precision_res["precision_20"]), np.mean(precision_res["precision_50"])))
print("[MF-AKBIPS-Exp] f1@20:{:.6f}, f1@50:{:.6f}".format(
        2 * (np.mean(precision_res["precision_20"]) * np.mean(recall_res["recall_20"])) / (np.mean(precision_res["precision_20"]) + np.mean(recall_res["recall_20"])), 
        2 * (np.mean(precision_res["precision_50"]) * np.mean(recall_res["recall_50"])) / (np.mean(precision_res["precision_50"]) + np.mean(recall_res["recall_50"]))))
print("***"*5 + "[MF-AKBIPS-Exp]" + "***"*5)

### WKBIPS

In [ ]:
mf_wkbips_gau = MF_WKBIPS_Gau(num_user, num_item)
mf_wkbips_gau.cuda()
ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]
mf_wkbips_gau.fit(x_train, y_train,  y_ips,
    lr1=3e-2,
    lamb1=1e-4,
    lr2=3e-2,
    lamb2=1e-4,
    lr3=3e-2,
    lamb3=1e-3,
    gamma = 1,
    batch_size=2048,
    num_w_epo = 3,
    J = 3,
    C = 1e-3,
    tol=1e-5)
test_pred = mf_wkbips_gau.predict(x_test)
mse_mfwkbips = mse_func(y_test, test_pred)
auc_mfwkbips = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_wkbips_gau, x_test, y_test, top_k_list = [20, 50])
recall_res = recall_func(mf_wkbips_gau, x_test, y_test, top_k_list = [20, 50])
precision_res = precision_func(mf_wkbips_gau, x_test, y_test, top_k_list = [20, 50])

print("***"*5 + "[MF-WKBIPS-Gau]" + "***"*5)
print("[MF-WKBIPS-Gau] test mse:", mse_mfwkbips)
print("[MF-WKBIPS-Gau] test auc:", auc_mfwkbips)
print("[MF-WKBIPS-Gau] ndcg@20:{:.6f}, ndcg@50:{:.6f}".format(
        np.mean(ndcg_res["ndcg_20"]), np.mean(ndcg_res["ndcg_50"])))
print("[MF-WKBIPS-Gau] recall@20:{:.6f}, recall@50:{:.6f}".format(
        np.mean(recall_res["recall_20"]), np.mean(recall_res["recall_50"])))
print("[MF-WKBIPS-Gau] precision@20:{:.6f}, precision@50:{:.6f}".format(
        np.mean(precision_res["precision_20"]), np.mean(precision_res["precision_50"])))
print("[MF-WKBIPS-Gau] f1@20:{:.6f}, f1@50:{:.6f}".format(
        2 * (np.mean(precision_res["precision_20"]) * np.mean(recall_res["recall_20"])) / (np.mean(precision_res["precision_20"]) + np.mean(recall_res["recall_20"])), 
        2 * (np.mean(precision_res["precision_50"]) * np.mean(recall_res["recall_50"])) / (np.mean(precision_res["precision_50"]) + np.mean(recall_res["recall_50"]))))
print("***"*5 + "[MF-WKBIPS-Gau]" + "***"*5)

In [ ]:
mf_wkbips_exp = MF_WKBIPS_Exp(num_user, num_item)
mf_wkbips_exp.cuda()
ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]
mf_wkbips_exp.fit(x_train, y_train,  y_ips,
    lr1=3e-2,
    lamb1=1e-4,
    lr2=3e-2,
    lamb2=1e-4,
    lr3=3e-2,
    lamb3=1e-3,
    gamma = 1,
    batch_size=2048,
    num_w_epo = 3,
    J = 3,
    C = 1e-3,
    tol=1e-5)
test_pred = mf_wkbips_exp.predict(x_test)
mse_mfwkbips = mse_func(y_test, test_pred)
auc_mfwkbips = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_wkbips_exp, x_test, y_test, top_k_list = [20, 50])
recall_res = recall_func(mf_wkbips_exp, x_test, y_test, top_k_list = [20, 50])
precision_res = precision_func(mf_wkbips_exp, x_test, y_test, top_k_list = [20, 50])

print("***"*5 + "[MF-WKBIPS-Exp]" + "***"*5)
print("[MF-WKBIPS-Exp] test mse:", mse_mfwkbips)
print("[MF-WKBIPS-Exp] test auc:", auc_mfwkbips)
print("[MF-WKBIPS-Exp] ndcg@20:{:.6f}, ndcg@50:{:.6f}".format(
        np.mean(ndcg_res["ndcg_20"]), np.mean(ndcg_res["ndcg_50"])))
print("[MF-WKBIPS-Exp] recall@20:{:.6f}, recall@50:{:.6f}".format(
        np.mean(recall_res["recall_20"]), np.mean(recall_res["recall_50"])))
print("[MF-WKBIPS-Exp] precision@20:{:.6f}, precision@50:{:.6f}".format(
        np.mean(precision_res["precision_20"]), np.mean(precision_res["precision_50"])))
print("[MF-WKBIPS-Exp] f1@20:{:.6f}, f1@50:{:.6f}".format(
        2 * (np.mean(precision_res["precision_20"]) * np.mean(recall_res["recall_20"])) / (np.mean(precision_res["precision_20"]) + np.mean(recall_res["recall_20"])), 
        2 * (np.mean(precision_res["precision_50"]) * np.mean(recall_res["recall_50"])) / (np.mean(precision_res["precision_50"]) + np.mean(recall_res["recall_50"]))))
print("***"*5 + "[MF-WKBIPS-Exp]" + "***"*5)

### KBDR

In [ ]:
mf_kbdr_gau = MF_KBDR_Gau(num_user, num_item)
mf_kbdr_gau.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_kbdr_gau.fit(x_train, y_train, y_ips, 
    lr=1e-2,
    lamb=1e-5,
    lr2=1e-2,
    lamb2=5e-2,
    gamma = 1,
    batch_size=2048,
    J = 3,
    C = 1e-2,
    tol=1e-5)

test_pred = mf_kbdr_gau.predict(x_test)
mse_mfkbdr = mse_func(y_test, test_pred)
auc_mfkbdr = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_kbdr_gau, x_test, y_test, top_k_list = [20, 50])
recall_res = recall_func(mf_kbdr_gau, x_test, y_test, top_k_list = [20, 50])
precision_res = precision_func(mf_kbdr_gau, x_test, y_test, top_k_list = [20, 50])

print("***"*5 + "[MF-KBDR-Gau]" + "***"*5)
print("[MF-KBDR-Gau] test mse:", mse_mfkbdr)
print("[MF-KBDR-Gau] test auc:", auc_mfkbdr)
print("[MF-KBDR-Gau] ndcg@20:{:.6f}, ndcg@50:{:.6f}".format(
        np.mean(ndcg_res["ndcg_20"]), np.mean(ndcg_res["ndcg_50"])))
print("[MF-KBDR-Gau] recall@20:{:.6f}, recall@50:{:.6f}".format(
        np.mean(recall_res["recall_20"]), np.mean(recall_res["recall_50"])))
print("[MF-KBDR-Gau] precision@20:{:.6f}, precision@50:{:.6f}".format(
        np.mean(precision_res["precision_20"]), np.mean(precision_res["precision_50"])))
print("[MF-KBDR-Gau] f1@20:{:.6f}, f1@50:{:.6f}".format(
        2 * (np.mean(precision_res["precision_20"]) * np.mean(recall_res["recall_20"])) / (np.mean(precision_res["precision_20"]) + np.mean(recall_res["recall_20"])), 
        2 * (np.mean(precision_res["precision_50"]) * np.mean(recall_res["recall_50"])) / (np.mean(precision_res["precision_50"]) + np.mean(recall_res["recall_50"]))))
print("***"*5 + "[MF-KBDR-Gau]" + "***"*5)

In [ ]:
mf_kbdr_exp = MF_KBDR_Exp(num_user, num_item)
mf_kbdr_exp.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_kbdr_exp.fit(x_train, y_train, y_ips, 
    lr=1e-2,
    lamb=2e-5,
    lr2=1e-2,
    lamb2=1e-1,
    gamma = 1,
    batch_size=2048,
    J = 3,
    C = 1e-2,
    tol=1e-5)

test_pred = mf_kbdr_exp.predict(x_test)
mse_mfkbdr = mse_func(y_test, test_pred)
auc_mfkbdr = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_kbdr_exp, x_test, y_test, top_k_list = [20, 50])
recall_res = recall_func(mf_kbdr_exp, x_test, y_test, top_k_list = [20, 50])
precision_res = precision_func(mf_kbdr_exp, x_test, y_test, top_k_list = [20, 50])

print("***"*5 + "[MF-KBDR-Exp]" + "***"*5)
print("[MF-KBDR-Exp] test mse:", mse_mfkbdr)
print("[MF-KBDR-Exp] test auc:", auc_mfkbdr)
print("[MF-KBDR-Exp] ndcg@20:{:.6f}, ndcg@50:{:.6f}".format(
        np.mean(ndcg_res["ndcg_20"]), np.mean(ndcg_res["ndcg_50"])))
print("[MF-KBDR-Exp] recall@20:{:.6f}, recall@50:{:.6f}".format(
        np.mean(recall_res["recall_20"]), np.mean(recall_res["recall_50"])))
print("[MF-KBDR-Exp] precision@20:{:.6f}, precision@50:{:.6f}".format(
        np.mean(precision_res["precision_20"]), np.mean(precision_res["precision_50"])))
print("[MF-KBDR-Exp] f1@20:{:.6f}, f1@50:{:.6f}".format(
        2 * (np.mean(precision_res["precision_20"]) * np.mean(recall_res["recall_20"])) / (np.mean(precision_res["precision_20"]) + np.mean(recall_res["recall_20"])), 
        2 * (np.mean(precision_res["precision_50"]) * np.mean(recall_res["recall_50"])) / (np.mean(precision_res["precision_50"]) + np.mean(recall_res["recall_50"]))))
print("***"*5 + "[MF-KBDR-Exp]" + "***"*5)

### AKBDR

In [ ]:
mf_akbdr_gau = MF_AKBDR_Gau(num_user, num_item)
mf_akbdr_gau.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_akbdr_gau.fit(x_train, y_train, y_ips, 
    lr=3e-2,
    lamb=1e-6,
    lr1=3e-2,
    lamb1=1e-4,
    lr2=3e-2,
    lamb2=1e-1,
    lr3=1e-2,
    lamb3=1e-6,
    gamma=5,
    batch_size=2048,
    num_w_epo = 3,
    J = 3,
    C = 1e-6,
    tol=1e-5)


test_pred = mf_akbdr_gau.predict(x_test)
mse_mfakbdr = mse_func(y_test, test_pred)
auc_mfakbdr = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_akbdr_gau, x_test, y_test, top_k_list = [20, 50])
recall_res = recall_func(mf_akbdr_gau, x_test, y_test, top_k_list = [20, 50])
precision_res = precision_func(mf_akbdr_gau, x_test, y_test, top_k_list = [20, 50])

print("***"*5 + "[MF-AKBDR-Gau]" + "***"*5)
print("[MF-AKBDR-Gau] test mse:", mse_mfakbdr)
print("[MF-AKBDR-Gau] test auc:", auc_mfakbdr)
print("[MF-AKBDR-Gau] ndcg@20:{:.6f}, ndcg@50:{:.6f}".format(
        np.mean(ndcg_res["ndcg_20"]), np.mean(ndcg_res["ndcg_50"])))
print("[MF-AKBDR-Gau] recall@20:{:.6f}, recall@50:{:.6f}".format(
        np.mean(recall_res["recall_20"]), np.mean(recall_res["recall_50"])))
print("[MF-AKBDR-Gau] precision@20:{:.6f}, precision@50:{:.6f}".format(
        np.mean(precision_res["precision_20"]), np.mean(precision_res["precision_50"])))
print("[MF-AKBDR-Gau] f1@20:{:.6f}, f1@50:{:.6f}".format(
        2 * (np.mean(precision_res["precision_20"]) * np.mean(recall_res["recall_20"])) / (np.mean(precision_res["precision_20"]) + np.mean(recall_res["recall_20"])), 
        2 * (np.mean(precision_res["precision_50"]) * np.mean(recall_res["recall_50"])) / (np.mean(precision_res["precision_50"]) + np.mean(recall_res["recall_50"]))))
print("***"*5 + "[MF-AKBDR-Gau]" + "***"*5)

In [ ]:
mf_akbdr_exp = MF_AKBDR_Exp(num_user, num_item)
mf_akbdr_exp.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_akbdr_exp.fit(x_train, y_train, y_ips, 
    lr=3e-2,
    lamb=1e-6,
    lr1=3e-2,
    lamb1=1e-4,
    lr2=3e-2,
    lamb2=1e-1,
    lr3=1e-2,
    lamb3=1e-6,
    gamma=1,
    batch_size=2048,
    num_w_epo = 3,
    J = 3,
    C = 1e-6,
    tol=1e-5)

test_pred = mf_akbdr_exp.predict(x_test)
mse_mfakbdr = mse_func(y_test, test_pred)
auc_mfakbdr = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_akbdr_exp, x_test, y_test, top_k_list = [20, 50])
recall_res = recall_func(mf_akbdr_exp, x_test, y_test, top_k_list = [20, 50])
precision_res = precision_func(mf_akbdr_exp, x_test, y_test, top_k_list = [20, 50])

print("***"*5 + "[MF-AKBDR-Exp]" + "***"*5)
print("[MF-AKBDR-Exp] test mse:", mse_mfakbdr)
print("[MF-AKBDR-Exp] test auc:", auc_mfakbdr)
print("[MF-AKBDR-Exp] ndcg@20:{:.6f}, ndcg@50:{:.6f}".format(
        np.mean(ndcg_res["ndcg_20"]), np.mean(ndcg_res["ndcg_50"])))
print("[MF-AKBDR-Exp] recall@20:{:.6f}, recall@50:{:.6f}".format(
        np.mean(recall_res["recall_20"]), np.mean(recall_res["recall_50"])))
print("[MF-AKBDR-Exp] precision@20:{:.6f}, precision@50:{:.6f}".format(
        np.mean(precision_res["precision_20"]), np.mean(precision_res["precision_50"])))
print("[MF-AKBDR-Exp] f1@20:{:.6f}, f1@50:{:.6f}".format(
        2 * (np.mean(precision_res["precision_20"]) * np.mean(recall_res["recall_20"])) / (np.mean(precision_res["precision_20"]) + np.mean(recall_res["recall_20"])), 
        2 * (np.mean(precision_res["precision_50"]) * np.mean(recall_res["recall_50"])) / (np.mean(precision_res["precision_50"]) + np.mean(recall_res["recall_50"]))))
print("***"*5 + "[MF-AKBDR-Exp]" + "***"*5)

### WKBDR

In [ ]:
mf_wkbdr_gau = MF_WKBDR_Gau(num_user, num_item)
mf_wkbdr_gau.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]
mf_wkbdr_gau.fit(x_train, y_train,  y_ips,
    lr=1e-2,
    lamb=1e-5,
    lr2=1e-2,
    lamb2=1e-1,
    gamma=5,
    batch_size=2048,
    J = 3,
    C = 1e-2,
    tol=1e-5)

test_pred = mf_wkbdr_gau.predict(x_test)
mse_mfwkbdr = mse_func(y_test, test_pred)
auc_mfwkbdr = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_wkbdr_gau, x_test, y_test, top_k_list = [20, 50])
recall_res = recall_func(mf_wkbdr_gau, x_test, y_test, top_k_list = [20, 50])
precision_res = precision_func(mf_wkbdr_gau, x_test, y_test, top_k_list = [20, 50])

print("***"*5 + "[MF-WKBDR-Gau]" + "***"*5)
print("[MF-WKBDR-Gau] test mse:", mse_mfwkbdr)
print("[MF-WKBDR-Gau] test auc:", auc_mfwkbdr)
print("[MF-WKBDR-Gau] ndcg@20:{:.6f}, ndcg@50:{:.6f}".format(
        np.mean(ndcg_res["ndcg_20"]), np.mean(ndcg_res["ndcg_50"])))
print("[MF-WKBDR-Gau] recall@20:{:.6f}, recall@50:{:.6f}".format(
        np.mean(recall_res["recall_20"]), np.mean(recall_res["recall_50"])))
print("[MF-WKBDR-Gau] precision@20:{:.6f}, precision@50:{:.6f}".format(
        np.mean(precision_res["precision_20"]), np.mean(precision_res["precision_50"])))
print("[MF-WKBDR-Gau] f1@20:{:.6f}, f1@50:{:.6f}".format(
        2 * (np.mean(precision_res["precision_20"]) * np.mean(recall_res["recall_20"])) / (np.mean(precision_res["precision_20"]) + np.mean(recall_res["recall_20"])), 
        2 * (np.mean(precision_res["precision_50"]) * np.mean(recall_res["recall_50"])) / (np.mean(precision_res["precision_50"]) + np.mean(recall_res["recall_50"]))))
print("***"*5 + "[MF-WKBDR-Gau]" + "***"*5)

In [ ]:
mf_wkbdr_exp = MF_WKBDR_Exp(num_user, num_item)
mf_wkbdr_exp.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]
mf_wkbdr_exp.fit(x_train, y_train,  y_ips,
    lr=1e-2,
    lamb=1e-6,
    lr2=1e-2,
    lamb2=1e-1,
    gamma=5,
    batch_size=2048,
    J = 3,
    C = 1e-2,
    tol=1e-5)

test_pred = mf_wkbdr_exp.predict(x_test)
mse_mfwkbdr = mse_func(y_test, test_pred)
auc_mfwkbdr = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_wkbdr_exp, x_test, y_test, top_k_list = [20, 50])
recall_res = recall_func(mf_wkbdr_exp, x_test, y_test, top_k_list = [20, 50])
precision_res = precision_func(mf_wkbdr_exp, x_test, y_test, top_k_list = [20, 50])

print("***"*5 + "[MF-WKBDR-Exp]" + "***"*5)
print("[MF-WKBDR-Exp] test mse:", mse_mfwkbdr)
print("[MF-WKBDR-Exp] test auc:", auc_mfwkbdr)
print("[MF-WKBDR-Exp] ndcg@20:{:.6f}, ndcg@50:{:.6f}".format(
        np.mean(ndcg_res["ndcg_20"]), np.mean(ndcg_res["ndcg_50"])))
print("[MF-WKBDR-Exp] recall@20:{:.6f}, recall@50:{:.6f}".format(
        np.mean(recall_res["recall_20"]), np.mean(recall_res["recall_50"])))
print("[MF-WKBDR-Exp] precision@20:{:.6f}, precision@50:{:.6f}".format(
        np.mean(precision_res["precision_20"]), np.mean(precision_res["precision_50"])))
print("[MF-WKBDR-Exp] f1@20:{:.6f}, f1@50:{:.6f}".format(
        2 * (np.mean(precision_res["precision_20"]) * np.mean(recall_res["recall_20"])) / (np.mean(precision_res["precision_20"]) + np.mean(recall_res["recall_20"])), 
        2 * (np.mean(precision_res["precision_50"]) * np.mean(recall_res["recall_50"])) / (np.mean(precision_res["precision_50"]) + np.mean(recall_res["recall_50"]))))
print("***"*5 + "[MF-WKBDR-Exp]" + "***"*5)